# **References**

https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-train-lb-0-361

https://www.kaggle.com/rhythmcam/ast-basic-string-expression

https://www.kaggle.com/vexxingbanana/sartorius-coco-dataset-notebook

https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/293723

# This Coco Json File Format is useful for training models with [MMTracking](http://github.com/open-mmlab/mmtracking)

# **Import Libraries**

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json
import glob
import random
import cv2
import tqdm
from tqdm import tqdm
import re
import ast

# **Helper Functions**

In [ ]:
def get_boxes(row):
    """Return the bboxes for a given row as a 3D matrix """
    #if len(row['annotations']) == 0:
    #    row['annotations'] = [{'x': -1, 'y': -1, 'width': -1, 'height': -1}]
    return pd.DataFrame(row['annotations'], columns=['x', 'y', 'width', 'height']).astype(float).values

# **Create CocoVid Json File**

In [ ]:
train_df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')

In [ ]:
train_df

In [ ]:
output_json_dict = {
    "images": [],
    "videos": [],
    "annotations": [],
    "categories": []
}
category_dict = {"id": 1, "name": "starfish", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)
video_0_dict = {"name": "video_0", "id": 0}
output_json_dict["videos"].append(video_0_dict)
video_1_dict = {"name": "video_1", "id": 1}
output_json_dict["videos"].append(video_1_dict)
# video_2_dict = {"name": "video_2", "id": 2}
# output_json_dict["videos"].append(video_2_dict)

In [ ]:
annot_id = 0
img_id = 0
video_0_frames = 0
video_1_frames = 0

In [ ]:
for f in tqdm(train_df.itertuples(), total=len(train_df)):
    if f[1] != 2:
        img_path = '../input/tensorflow-great-barrier-reef/train_images/video_' + str(f[1]) + '/' + f[5].split('-')[1] + '.jpg'
        img = cv2.imread(img_path)
        height, width, channels = img.shape
#         frame_id = f[3]
        video_id = f[1]
        if video_id == 0:
            frame_id = video_0_frames
        else:
            frame_id = video_1_frames

        img_info = {
            "id": f[0],
            "width": width,
            "height": height,
            "file_name": img_path,
            "frame_id": frame_id,
            "video_id": video_id,
        }
        output_json_dict["images"].append(img_info)
        if f[6] != '[]':
    #         print(train_df.iloc[f[0]]['annotations'])
            bbox_list = ast.literal_eval(f[6])
            for bbox in bbox_list:
    #             bbox = ast.literal_eval(bbox)
                if bbox['height'] + bbox['y'] > 720:
                    bbox['height'] = 720 - bbox['y']
                annot = {
                    "category_id": 1,
                    "bbox": [bbox['x'], bbox['y'], bbox['width'], bbox['height']],
                    "id": annot_id,
                    "image_id": f[0],
                    "area": bbox['width'] * bbox['height'],
                    "segmentation": [],
                    "iscrowd": 0,
                    "video_id": video_id,
                }
                output_json_dict["annotations"].append(annot)
                annot_id += 1
        img_id += 1
        if video_id == 0:
            video_0_frames += 1
        else:
            video_1_frames += 1

In [ ]:
with open('train_dataset.json', 'w') as f:
    output_json = json.dumps(output_json_dict)
    f.write(output_json)

In [ ]:
output_json_dict = {
    "images": [],
    "videos": [],
    "annotations": [],
    "categories": []
}
category_dict = {"id": 1, "name": "starfish", "supercategory": "none"}
output_json_dict["categories"].append(category_dict)
# video_0_dict = {"name": "video_0", "id": 0}
# output_json_dict["videos"].append(video_0_dict)
# video_1_dict = {"name": "video_1", "id": 1}
# output_json_dict["videos"].append(video_1_dict)
video_2_dict = {"name": "video_2", "id": 2}
output_json_dict["videos"].append(video_2_dict)

In [ ]:
annot_id = 0
img_id = 0
video_2_frames = 0

In [ ]:
for f in tqdm(train_df.itertuples(), total=len(train_df)):
    if f[1] == 2:
        img_path = '../input/tensorflow-great-barrier-reef/train_images/video_' + str(f[1]) + '/' + f[5].split('-')[1] + '.jpg'
        img = cv2.imread(img_path)
        height, width, channels = img.shape
        frame_id = video_2_frames
        video_id = f[1]

        img_info = {
            "id": img_id,
            "width": width,
            "height": height,
            "file_name": img_path,
            "frame_id": frame_id,
            "video_id": video_id
        }
        output_json_dict["images"].append(img_info)
        if f[6] != '[]':
    #         print(train_df.iloc[f[0]]['annotations'])
            bbox_list = ast.literal_eval(f[6])
            for bbox in bbox_list:
    #             bbox = ast.literal_eval(bbox)
                if bbox['height'] + bbox['y'] > 720:
                    bbox['height'] = 720 - bbox['y']
                annot = {
                    "category_id": 1,
                    "bbox": [bbox['x'], bbox['y'], bbox['width'], bbox['height']],
                    "id": annot_id,
                    "image_id": img_id,
                    "area": bbox['width'] * bbox['height'],
                    "segmentation": [],
                    "iscrowd": 0,
                    "video_id": video_id,
                }
                output_json_dict["annotations"].append(annot)
                annot_id += 1
        img_id += 1
        video_2_frames += 1

In [ ]:
with open('val_dataset.json', 'w') as f:
    output_json = json.dumps(output_json_dict)
    f.write(output_json)